# jupyterlab

> `jupyterlab` is both the high-level building blocks of Jupyter clients, as well as the reference implementation of a composable, IDE-like environment. It also has the most involved build and distribution setup.

In [ ]:
GH_ORG = NAME = "jupyterlab"
JS_NS = "@jupyterlab"
URL = f"https://github.com/{GH_ORG}/{NAME}"

In [ ]:
from pathlib import Path

import importnb

with importnb.Notebook():
    from jupyak.tasks import (
        __ipykernel,
        __jupyterlab_server,
        __lumino,
        __notebook_shim,
    )
    from jupyak.tasks import _actions as A
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y

## schema

In [ ]:
bu = "buildutils"
br = "builder"
bu_tsbi = f"{bu}/{W.TSCONFIG_BUILDINFO}"
br_tsbi = f"{br}/{W.TSCONFIG_BUILDINFO}"
br_lib_js = f"{br}/lib/build-labextension.js"
m_tsbi = f"packages/metapackage/{W.TSCONFIG_BUILDINFO}"
dm = "dev_mode"
dm_static_pj = f"{dm}/static/{W.PACKAGE_JSON}"
JS_ROOT_TASKS = [
    {
        "name": "buildutils",
        "actions": [["yarn", "build:utils"]],
        "file_dep": [f"{bu}/src/**/*.ts", f"{bu}/{W.PACKAGE_JSON}"],
        "targets": [bu_tsbi],
    },
    {
        "name": "builder",
        "actions": [["yarn", "build:builder"]],
        "file_dep": [f"{br}/src/**/*.ts", f"{br}/{W.PACKAGE_JSON}"],
        "targets": [br_tsbi, br_lib_js],
    },
]
JS_META_TASKS = [
    {
        "name": "build",
        "actions": [["yarn", "build"]],
        "file_dep": [
            f"../lumino/packages/*/{W.TSCONFIG_BUILDINFO}",
            "packages/*/src/**/*.ts",
            "packages/*/src/**/*.tsx",
        ],
        "targets": [m_tsbi],
    },
]
JS_DEV_TASKS = [
    {
        "name": "build:prod",
        "actions": [["yarn", "build:prod"]],
        "file_dep": [
            br_tsbi,
            bu_tsbi,
            m_tsbi,
            "packages/*/schema/*.json",
            "packages/*/style/**/*.css",
            "packages/*/style/**/*.svg",
            "packages/*/style/**/*.js",
        ],
        "targets": [dm_static_pj],
    },
]
JS_DIST_TASKS = {
    "./": JS_ROOT_TASKS,
    "./packages/metapackage/": JS_META_TASKS,
    "./dev_mode/": JS_DEV_TASKS,
}

In [ ]:
DEFAULTS = {
    "name": NAME,
    "github": {"url": URL},
    "js": {
        "dependencies": [
            __lumino.NAME,
        ],
        "tasks": JS_DIST_TASKS,
        "dist_exclude_patterns": [
            "/galata",
            "/examples",
            "/dev_mode",
            "/metapackage",
            "/mock",
        ],
    },
    "py": {
        "dependencies": [
            __jupyterlab_server.NAME,
            __ipykernel.NAME,
            __notebook_shim.NAME,
            "jupyter_lsp",
            "jupyterlab_pygments",
        ],
        "file_dep": [
            dm_static_pj,
        ],
    },
}

> technically, the `jupyterlab` python package depends on `jupyter_lsp`, but this doesn't block building the JupyterLite site. See more in the [`jupyterlab_lsp` notebook](./075_jupyterlab_lsp.ipynb).

In [ ]:
CLEAN_REPO_DEPS = tuple(
    [p for p in DEFAULTS["py"]["dependencies"] if p != "jupyter_lsp"]
    + DEFAULTS["js"]["dependencies"]
)

In [ ]:
@Y.Yak.repo(NAME, CLEAN_REPO_DEPS)
def default_factory(yak: Y.Yak):
    return DEFAULTS

## tasks

> some custom tasks are needed to fool the rest of the build chain

In [ ]:
def task_sweep():
    with importnb.Notebook():
        from jupyak.tasks import _shave as S

    yak = S._ensure_yak()
    repo = yak.repos["jupyterlab"]
    work_dir, in_repo = repo.run_context

    subdirs = ["static", "themes", "schemas"]
    dev_mode = work_dir / "dev_mode"
    dests = [yak.env.lab_share, work_dir / "jupyterlab/staging"]

    yield dict(
        name="assets",
        actions=[
            (A.copy, [dev_mode / subdir, dest / subdir])
            for subdir in subdirs
            for dest in dests
        ],
        file_dep=[work_dir / f"dev_mode/static/{W.PACKAGE_JSON}"],
        targets=[*[dest / "static" / W.PACKAGE_JSON for dest in dests]],
    )

    br_lib_js_path = repo.work_path / br_lib_js
    script_path = repo.py.labextension_script

    yield dict(
        name="labextension",
        doc="> create a patched labextension script",
        file_dep=[br_lib_js_path],
        targets=[script_path],
        actions=[
            (_write_labextension_script, [script_path, br_lib_js_path]),
        ],
    )

> some custom tasks are needed to fool the rest of the build chain

## utilities 

In [ ]:
def _write_labextension_script(dest: Path, lib_js: Path):
    dest.parent.mkdir(parents=True, exist_ok=True)
    dest.write_text(
        f"""
import sys
from jupyterlab import federated_labextensions
from jupyterlab.labextensions import LabExtensionApp
federated_labextensions._ensure_builder = lambda *_: "{lib_js}"

main = LabExtensionApp.launch_instance

if __name__ == "__main__":
    sys.exit(main())
""",
    )